# <center>TRABAJO FINAL</center>

- **Curso:** Lenguaje de Programación II.
- **Semestre:** 2021-2
- **Tema:** Webscrapeo a UniProt (https://www.uniprot.org/).

In [3]:
#Importamos las librerías necesarias para el webscrapeo
from bs4 import BeautifulSoup #Para el webscrapeo
import requests               #Para la lectura de la página
import pandas as pd           #Para las tablas
import re                     #Para usar expresiones regulares

In [4]:
#Leemos las hojas del excel y seleccionamos la columnas que contienen 'Gen Abrev'
trabajo = pd.read_excel("Genes identificados con vecinos.xlsx",sheet_name=None) #Leemos todas las hojas
lista=[] #Se crea la lista 'lista' 

for x in trabajo:
    lista+=list(trabajo[x].iloc[:,2]) #Por cada hoja se leen las filas de la posición 2 y se almacenan en 'lista'
GENES=[] #Se crea la lista 'GENES'
a=0

#Se asignan a la lista 'GENES' todas las filas que no tienen más de 30 carácteres, están vacías o tienen la etiqueta 'Gen Abrev'
for y in lista:
    if len(str(y))>30 or str(y)=='nan' or y=='Gen Abrev': 
        pass
    else:
        GENES.append(y) 

In [5]:
GENES

['TRNAG-UCC',
 'ROBO1',
 'TRNAC-GCA',
 'TRNAC-ACA',
 'MIR2286',
 'LOC101910021',
 'LOC112447295',
 'CD47',
 'IFT57',
 'HHLA2',
 'MYH15',
 'CIP2A',
 'DZIP3',
 'TRAT1',
 'MORC1',
 'C1H3orf85',
 'LOC112448290',
 'LOC101903052',
 'DPPA2',
 'DPPA4',
 'LOC100337136',
 'LOC783844',
 'RTP4',
 'MASP1',
 'RTP1',
 'LOC100847189',
 'ST6GAL1',
 'MIR2285K-3',
 'ADIPOQ',
 'RFC4',
 'EIF4A2',
 'MIR1248-1',
 'KNG1',
 'HRG',
 'FETUB',
 'AHSG',
 'LOC101906884',
 'DNAJB11',
 'TBCCD1',
 'CRYGS',
 'DGKG',
 'ETV5',
 'LOC616971',
 'TRA2B',
 'TRNAE-UUC',
 'IGF2BP2',
 'SENP2',
 'LIPH',
 'TMEM41A',
 'MAP3K13',
 'EHHADH',
 'EPHB1',
 'KY',
 'CEP63',
 'ANAPC13',
 'AMOTL2',
 'RYK',
 'SLCO2A1',
 'RAB6B',
 'TF',
 'LOC525947',
 'TOPBP1',
 'RIPK4',
 'PRDM15',
 'C2CD2',
 'ZBTB21',
 'UMODL1',
 'ABCG1',
 'TFF3',
 'TFF2',
 'TFF1',
 'TMPRSS3',
 'UBASH3A',
 'RSPH1',
 'SLC37A1',
 'PDE9A',
 'WDR4',
 'NDUFV3',
 'PKNOX1',
 'CBS',
 'U2AF1',
 'CRYAA',
 'SIK1',
 'HSF2BP',
 'RRP1B',
 'PDXK',
 'CSTB',
 'RRP1',
 'AGPAT3',
 'KCNH7',
 'GC

In [6]:
len(GENES)

2531

In [ ]:
#Se crean las etiquetas de cada columna del webscrapeo en 'Gen_Bos_Taurus'
Gen_Bos_Taurus=[['GEN', 'entry', 'entryname' , 'protein', 'gene', 'organism', 'status', 'GO', 'Molecular_function', 'Biological_process', 'Ligand']]

for GEN in GENES: 
    url = (f"https://www.uniprot.org/uniprot/?query={GEN}+bos+taurus&sort=score") #Url con cada nombre del gen y la palabra Bos Taurus
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    
#Si por cada nombre del gen se encuentra el mensaje 'Sorry, no results found for your search term' en el texto de la página 
#se asignan vacíos ya que la url no muestra resultados para webscrapear
    if 'Sorry, no results found for your search term.' in str(soup.text):
        caso=[GEN, '', '' , '', '', '', '', '', '', '','']
        
        

    else: #En cambio si por cada nombre del gen se encuentran resultados:

        #Encontrando entry
        #Encuentra la primera etiqueta <tr> con la clase "entry selected-row" y extrae el texto de la referencia
        entry=soup.find('tr',class_="entry selected-row").a.text 


        #Encontrando entryname
        #Encuentra la primera etiqueta <tr> con la clase "entry selected-row", luego busca todas las etiquetas <td>  
        #pero solo extrae el texto de la que está en la posición 2
        entryname=soup.find('tr',class_="entry selected-row").find_all('td')[2].text

        #Ahora trabajando con entry para encontrar protein, gene, organism, status, GO, Molecular function, Biological process y Ligand 
        url2= (f"https://www.uniprot.org/uniprot/{entry}") #Url con cada nombre del entry
        html2 = requests.get(url2)
        soup2 = BeautifulSoup(html2.content, 'html.parser')

        #Encuentra la primera etiqueta <div> con id 'content-protein' y extrae el texto
        protein=soup2.find('div', id='content-protein').text
        #Encuentra la primera etiqueta <div> con id 'content-gene' y extrae el texto
        gene=soup2.find('div', id='content-gene').text
        #Encuentra la primera etiqueta <div> con id 'content-organism' y extrae el texto
        organism=soup2.find('div', id='content-organism').text
        #Encuentra la primera etiqueta <div> con id 'content-status' y extrae el texto de la primera etiqueta <span>
        status=soup2.find('div',id="content-status").span.text

        #Encuentra la primera etiqueta <ul> con la clase "noNumbering biological_process"
        GO1=soup2.find('ul', class_="noNumbering biological_process")
        #Si no encuentra nada en GO1 se colocara como una cadena vacia
        if GO1 is None:
            GO=''
        #De lo contrario, si encuentra resultados
        else:
            #Si encuentra, va a buscar todas las etiquetas <a> y que termine en 'Display-GO-Term'
            GO2=GO1.find_all('a', onclick="window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');")
            GO=''
            #Se hace un bucle para apendizar el texto de cada GO2 encontrado
            for x in GO2:
                GO+=x.text+'. '
        
        #Encuentra la primera etiqueta <div>, con la clase "section" y con id "function"
        funcion=soup2.find('div', class_="section", id="function")
        #Si no encuentra nada, llenara cada funcion como cadenas vacias
        if funcion is None:
            Molecular_function=''
            Biological_process=''
            Ligand=''
        #De el contrario, si encuentra resultados
        else:
            #Si no encuentra la palabra 'Keywords' dentro de la cadena funcion 
            if 'Keywords' not in str(funcion):
                Molecular_function=''
                Biological_process=''
                Ligand=''
            #De lo contrario, si encuentra la palabra 'Keywords'
            else:
                #Encuentra la primera etiqueta <table> con la clase "databaseTable"
                key1=funcion.find('table', class_="databaseTable")
                #Si no encuentra nada en la key1, se completa con cadenas vacias
                if key1 is None:
                    Molecular_function=''
                    Biological_process=''
                    Ligand=''
                #De lo contrario, si encuentra resultados
                else:
                    #Que encuentre todas las etiquetas <tr>
                    key=key1.find_all('tr')  
                    
                    #Se crea una lista llamada llave y otra llamada clave
                    llave=[]
                    clave=[]
                    #Para cada uno de los elementos de "key" 
                    for x in key:
                        #Va a encontrar el texto de la primera etiqueta <td>
                        primero=x.find('td').text
                        #Se almacenara en la lista llave
                        llave.append(primero)
                        for y in x:
                            #Se crea una cadena vacia
                            lista_datos=''
                            #Se encuentran todas las etiquetas <a> y se usa una expresion regular "href=re.compile("/keywords/")"
                            datos=y.find_all('a',href=re.compile("/keywords/"))        
                            for z in datos:
                                #Se usa para unir cada uno de los datos a una cadena de texto
                                lista_datos+=z.text+'. '
                            #Si no encuentra nada en lista_datos pasara
                            if lista_datos=='':
                                pass
                            #De lo contrario, agregara lista_datos a la lista clave
                            else:
                                clave.append(lista_datos)
                    #Si encuentra funcion molecular en llave
                    if 'Molecular function' in llave:
                        #Va a buscar el indice de la llave, para que clave tenga el mismo indice 
                        Molecular_function=clave[llave.index('Molecular function')]
                    #De lo contrario,se completara con un espacio vacio
                    else:
                        Molecular_function=''
                    #Si encuentra biological process en llave
                    if 'Biological process' in llave:
                        #Va a buscar el indice de la llave, para que clave tenga el mismo indice
                        Biological_process=clave[llave.index('Biological process')]
                    #De lo contrario,se completara con un espacio vacio
                    else:
                        Biological_process=''
                    #Si encuentra Ligand en llave
                    if 'Ligand' in llave:
                        #Va a buscar el indice de la llave, para que clave tenga el mismo indice
                        Ligand=clave[llave.index('Ligand')]
                    #De lo contrario,se completara con un espacio vacio
                    else:
                        Ligand=''
        #Por cada Gen que encuentre va a agregar una lista con todos los elementos requeridos
        caso=[GEN, entry, entryname , protein, gene, organism, status, GO, Molecular_function, Biological_process, Ligand]
    Gen_Bos_Taurus.append(caso)
#Se convierte en un dataFrame
df = pd.DataFrame(Gen_Bos_Taurus)
#Se convierte a excel
df.to_excel('Genes-Bos-Taurus.xlsx', header=False,index=False)   
    



## Integrantes:

- Charun Gameros, Rodrigo 
- Chumbimuni Ricci, Roxana 
- Cuadros Ramirez, Kevin William